# DSC PJATK Car Price Prediction — Final Presentation

**Cyprian i Jakub:**  
- Grupa 7

## 1. Wprowadzenie
Witamy w naszej prezentacji rozwiązania konkursowego organizowanego przez **DSC PJATK**. Naszym zadaniem było przygotowanie **modelu predykcyjnego** do przewidywania cen pojazdów na podstawie danych z ofert sprzedaży.

**Kluczowe etapy pracy:**
1. Eksploracyjna analiza danych (EDA)
2. Czyszczenie i inżynieria cech (feature engineering)
3. Budowa i ocena modelu
4. Wnioski i rekomendacje

# A. Analiza i Wizualizacje (EDA + Wnioski)

## 2. Eksploracyjna Analiza Danych
W tej części koncentrujemy się na:
- Wczytaniu danych
- Podstawowych statystykach opisowych
- Identyfikacji braków w danych i wartości odstających
- Prostej wizualizacji rozkładów i zależności

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

sns.set(style="whitegrid")

# Wczytanie danych
df_train = pd.read_csv("Cleaned_train.csv", index_col="ID")  # przykładowa nazwa
df_test  = pd.read_csv("Cleaned_test.csv", index_col="ID")   # przykładowa nazwa
print("Rozmiary zbioru treningowego:", df_train.shape)
df_train.head()